# Biology experiment 3 - Leech Action Potential
## - Download recordings data files from Github

In [ ]:
import os

if os.path.basename(os.getcwd())!='Leech-Action-Potential-Colab':
  !git clone https://github.com/chenziao/Leech-Action-Potential-Colab.git
  %cd Leech-Action-Potential-Colab/
else:
  print('Data files have already been downloaded.')
%ls

#### Run the cell below. Select the file of the recording data in the menu and click on "Select" button. After selecting a recording, run the next cell below to plot the recording and start analysis.


In [ ]:
import glob
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display, clear_output

datafiles = sorted(glob.glob('*.txt'))

def select_files(*arg):
    global file_name
    file_name = w_file.value
    clear_output(wait=True)
    display(VBox([w_file,SelectFileButton]))
    print(file_name+' is selected.')

w_file = widgets.Dropdown(options=datafiles,description='Select Data:')
SelectFileButton = widgets.Button(description='Select',button_style='primary')
SelectFileButton.on_click(select_files)
select_files()

## - Plot Intracellular Recordings
#### Use the slider below the figure to select a time range to display. You can also enter the range directly in the text box.
#### Click on "Low-pass Filter" button to filter out the oscillatory noise arised from the internal circuit of the amplifier. Use the slider on the right to set the cut-off frequency. Lower cut-off frequency will reduce noise better but also attenuate the amplitude of spikes since spikes also have high frequency component. 400 Hz cut-off frequency is recommended if you are observing spikes. 200-250 Hz cut-off frequency is recommended when you are measuring passive properties.

### Note: When you answer the questions using the recording data, remember to specify which piece of data you use and show your work in questions involving calculation.

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

headers = ['V','I']
parameters = pd.read_csv(open(file_name),names=headers,nrows=3)
dt = [float(s) for s in parameters.iloc[0,0].split() if s.replace('.','',1).isdigit()][0] # time step (ms)
dV = [float(s) for s in parameters.iloc[1,0].split() if s.replace('.','',1).isdigit()][0] # Voltage unit (mV)
dI = [float(s) for s in parameters.iloc[2,0].split() if s.replace('.','',1).isdigit()][0] # Current unit (nA)
Fs = 1000/dt
data = pd.read_csv(open(file_name),names=headers,skiprows=4)
N = data.shape[0]
t = dt*np.arange(N)
T = t[-1]
V = dV*data['V'].values
I = dI*data['I'].values

freq_cutoff = 400
t_range = [0,T]
y_range = [0.,1.]

def plot_traces(filt,cutoff,t_range,y_range):
    [x1,x2] = [max(np.floor(t_range[0]/T*N).astype(int),0) , min(np.ceil(t_range[1]/T*N).astype(int),N)+1]
    plt.close('all')
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    if filt:
        ax1.plot(t[x1:x2],V[x1:x2],'cyan')
        ax1.plot(t[x1:x2],V_filt[x1:x2],'blue')
        ax1.legend(['Raw V','Filtered V'],loc=1)
        ax2.plot(t[x1:x2],I[x1:x2],'pink')
        ax2.plot(t[x1:x2],I_filt[x1:x2],'red')
        ax2.legend(['Raw I','Filtered I'],loc=1)
    else:
        ax1.plot(t[x1:x2],V[x1:x2],'b')
        ax2.plot(t[x1:x2],I[x1:x2],'r')
    ax1.set_xlim(t_range)
    ax2.set_xlim(t_range)
    yl = ax1.get_ylim()
    ax1.set_ylim(np.asarray(y_range)*(yl[1]-yl[0])+yl[0])
    ax1.set_ylabel('mV')
    ax2.set_ylabel('nA')
    ax2.set_xlabel('Time (ms)')
    plt.show()

w_reset = widgets.Button(description='Reset',icon='history',button_style='primary')
w_filt = widgets.ToggleButton(value=False,description='Low-pass Filter',button_style='info',icon='play')
w_cutoff = widgets.FloatSlider(value=freq_cutoff,min=10,max=Fs/2,step=1,disabled=True,continuous_update=False)
w_t_range = widgets.FloatRangeSlider(value=t_range,min=0,max=T,step=1,continuous_update=False,readout_format='.0f',layout=Layout(width='60%'))
w_y = widgets.FloatRangeSlider(value=y_range,min=0.,max=1.,step=0.01,readout=False,continuous_update=False,orientation='vertical',layout=Layout(height='200px'))

def reset_default(*args):
    w_filt.value = False
    w_cutoff.value = freq_cutoff
    w_y.value = y_range
    w_t_range.value = t_range
w_reset.on_click(reset_default)

def switch_filt(*arg):
    if w_filt.value:
        w_cutoff.disabled = False
    else:
        w_cutoff.disabled = True
w_filt.observe(switch_filt,'value')

def set_cutoff(*arg):
    global V_filt,I_filt
    bFilt,aFilt = signal.butter(2,w_cutoff.value*2/Fs,btype='lowpass')
    V_filt = signal.filtfilt(bFilt,aFilt,V)
    I_filt = signal.filtfilt(bFilt,aFilt,I)
w_cutoff.observe(set_cutoff,'value')

def set_y_range(*arg):
    min_dist = 0.05
    y_range = list(w_y.value)
    y_range[1] = min(max(y_range[0]+min_dist,y_range[1]),1.)
    y_range[0] = max(min(y_range[1]-min_dist,y_range[0]),0.)
    w_y.value = y_range
w_y.observe(set_y_range,'value')

set_cutoff()
out = interactive_output(plot_traces,{'filt':w_filt,'cutoff':w_cutoff,'t_range':w_t_range,'y_range':w_y})
ui = VBox([w_reset,HBox([out,VBox([w_y,Label('Zoom in')])]),HBox([Label('Display Time Range (ms):'),w_t_range]),HBox([w_filt,Label('Cut-off frequency'),w_cutoff])])
display(ui)